In [ ]:
from IPython.display import Markdown, display
from datetime import datetime

def signature():
    display(Markdown(f"""
---
### 👨‍💻 *Authored by [Erwin R. Pasia](mailto:erwinpasia@gmail.com)*
📅 Date: {datetime.now().strftime("%B %d, %Y")}

> *"Code is poetry. Simplicity is elegance."*  
---
"""))

signature()

In [1]:
# Autonomous GenAI Agent for Data Engineering v2.0 (Refactored for Capstone Project)

# This notebook implements a refactored version of the Data Engineering GenAI agent.
# The goal of this refactoring is to create a more modular, maintainable, and extensible structure,
# making the agent a powerful assistant for various stages of a Data Engineering Capstone Project.
# 
# **Key Refactoring Principles Applied:**
# - **Modularity:** Components (agent core, tools, knowledge base, config) are defined in separate conceptual modules (realized as distinct cells or groups of cells).
# - **Reusability:** Core logic and tools are designed for easier reuse.
# - **Configuration Management:** Settings are centralized.
# - **Clarity:** The structure aims to be easier to understand and navigate.


In [2]:
### 1. Setup - Install Dependencies

!pip install --upgrade --quiet "langgraph>=0.0.50" "langchain-google-genai>=1.0.3" \
"langchain-core>=0.1.40" "langchain-community>=0.0.38" "chromadb>=0.5.0" \
"google-api-python-client" "google-auth" "python-dotenv" \
"pandas>=2.0.0" "ipython"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 11.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 61.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 65.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.3/604.3 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
### 2. Imports - Standard Libraries & Initial Setup

import os
import json
import logging
import re
import datetime
import asyncio
import time
from typing import List, Dict, Any, Optional, TypedDict, Annotated, Literal

from IPython.display import Markdown, display, Image
import pandas as pd

# Attempt to load Google API key from Kaggle secrets or environment
_use_kaggle_secrets = False
GOOGLE_API_KEY = None

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    _use_kaggle_secrets = True
    print("Successfully loaded GOOGLE_API_KEY from Kaggle secrets.")
except Exception:
    print("Could not load GOOGLE_API_KEY from Kaggle secrets. Will try environment variable.")
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    print("WARNING: GOOGLE_API_KEY not found. Agent will not function correctly.")
else:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY # For LangChain
    # Configure google.generativeai directly if needed for embedding function
    try:
        import google.generativeai as genai
        genai.configure(api_key=GOOGLE_API_KEY)
        print("Google Generative AI SDK configured.")
    except ImportError:
        print("WARNING: google.generativeai package not found. RAG embeddings might fail.")
    except Exception as e:
        print(f"Error configuring Google Generative AI SDK: {e}")

# Max iterations for the agent
MAX_ITERATIONS_DE_CONFIG = 10


Successfully loaded GOOGLE_API_KEY from Kaggle secrets.
Google Generative AI SDK configured.


In [4]:
### 3. Logging Configuration

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')
logger = logging.getLogger('data_pipeline_agent_refactored')
logger.info("Logging setup complete for Refactored Data Pipeline Agent.")


In [5]:
### 4. Utilities (utils/config_loader.py & utils/helpers.py - Simulated)

# --- Simulating utils/config_loader.py ---
# In a real multi-file setup, this would load from .env or other config sources.
# Here, we primarily use constants defined in Cell 3 or directly.

def get_config() -> Dict[str, Any]:
    """Provides runtime configuration."""
    return {
        "GOOGLE_API_KEY": GOOGLE_API_KEY,
        "DB_PATH_DE_KB": "./chroma_db_data_engineering_kb_refactored",
        "COLLECTION_NAME_DE_KB": "data_engineering_knowledge_base_refactored",
        "LLM_MODEL_NAME": "gemini-2.0-flash", # Updated to a common Gemini model
        "LLM_TEMPERATURE": 0.2,
        "MAX_ITERATIONS_DE": MAX_ITERATIONS_DE_CONFIG,
        "EMBEDDING_MODEL_NAME": "models/text-embedding-004" # Specific embedding model
    }

# --- Simulating utils/helpers.py ---
def safe_json_loads(s: str, logger_instance=None) -> Optional[Dict[str, Any]]:
    """Safely loads a JSON string, logging errors if a logger is provided."""
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        if logger_instance:
            logger_instance.error(f"JSONDecodeError: {e} while parsing: {s[:100]}...")
        return None
    except TypeError as e: # Handle cases where s might not be a string
        if logger_instance:
            logger_instance.error(f"TypeError: {e} - input was not a string-like object for JSON parsing.")
        return None


logger.info("Utilities (config_loader, helpers) defined.")
AGENT_CONFIG = get_config() # Load config globally for easy access in notebook


In [6]:
### 5. Core Agent State (core/state.py - Simulated)

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages # add_messages handles appending to message lists

class DataPipelineAgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    initial_goal: str
    current_task_description: Optional[str]
    discovered_insights: List[Dict[str, Any]]
    generated_artifacts: List[Dict[str, Any]]
    pipeline_plan: List[str]
    executed_steps: List[Dict[str, Any]] # Storing more info about steps
    finished: bool
    iterations: int

logger.info("Core State (DataPipelineAgentState) defined.")


In [7]:
### 6. Knowledge Base Management (knowledge_base/kb_manager.py - Simulated)

import chromadb
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
# Corrected type import for Collection if needed, or use full path in hint
from chromadb.api.models.Collection import Collection as ChromaCollectionType # Alias for clarity
import google.generativeai as genai
from google.api_core import retry
from google.api_core.exceptions import ServiceUnavailable, ResourceExhausted, DeadlineExceeded, InternalServerError

# This should ideally be loaded from a file for better management (e.g., JSON, CSV)
# For this notebook, it's defined here.
CAPSTONE_KB_DATA = [
    {
        "id": "dq_rule_mysql_1", "type": "Data Quality Rule", "technology": "MySQL",
        "description": "Ensure primary keys in MySQL OLTP tables are not null and unique.",
        "suggestion": "Define columns as PRIMARY KEY. For existing data, check with `COUNT(DISTINCT pk_column) = COUNT(*)` and `SUM(IF(pk_column IS NULL, 1, 0)) = 0`."
    },
    {
        "id": "mongo_schema_flex", "type": "Schema Design", "technology": "MongoDB",
        "description": "MongoDB's flexible schema is ideal for product catalogs with varying attributes.",
        "suggestion": "Embed related product attributes. Use arrays for multi-value fields like tags or images."
    },
    {
        "id": "dwh_star_schema", "type": "DWH Design", "technology": "PostgreSQL/DB2",
        "description": "Star schema is recommended for retail DWH for sales analytics.",
        "suggestion": "Identify central fact table (e.g., sales transactions) and surrounding dimension tables (customer, product, time, store)."
    },
    {
        "id": "airflow_idempotency", "type": "ETL Best Practice", "technology": "Airflow",
        "description": "Airflow tasks should be idempotent, meaning running them multiple times with the same input yields the same result.",
        "suggestion": "Use INSERT...ON CONFLICT or staging tables with MERGE operations for DWH loads."
    },
    {
        "id": "kafka_topic_naming", "type": "Streaming", "technology": "Kafka",
        "description": "Consistent Kafka topic naming conventions improve manageability.",
        "suggestion": "Use a pattern like `<source_system>.<dataset_name>.<event_type>`, e.g., `mysql.orders.created`."
    },
    {
        "id": "sparkml_feature_scaling", "type": "Machine Learning", "technology": "SparkML",
        "description": "Feature scaling (e.g., normalization, standardization) is important for many SparkML algorithms.",
        "suggestion": "Use `StandardScaler` or `MinMaxScaler` from `pyspark.ml.feature`."
    }
]

class GeminiEmbeddingFunctionForChroma(EmbeddingFunction):
    def __init__(self, api_key: Optional[str] = None,
                 model_name: str = AGENT_CONFIG["EMBEDDING_MODEL_NAME"],
                 task_type: str = "retrieval_document"): # Default for adding docs
        super().__init__()
        if not api_key and not os.environ.get("GOOGLE_API_KEY"):
            raise ValueError("Google API Key must be provided or set as GOOGLE_API_KEY environment variable.")
        self.model_name = model_name
        self.task_type = task_type
        # genai.configure should have been called already if GOOGLE_API_KEY is set
        if not genai._global_config.api_key: # type: ignore
             if api_key:
                genai.configure(api_key=api_key)
             else: # Try to re-configure from env if not already set by global config
                env_api_key = os.environ.get("GOOGLE_API_KEY")
                if env_api_key:
                    genai.configure(api_key=env_api_key)
                else:
                    raise ValueError("genai SDK not configured and API key is missing.")


    @retry.Retry(
        predicate=retry.if_exception_type(ServiceUnavailable, ResourceExhausted, DeadlineExceeded, InternalServerError),
        initial=1.0, maximum=10.0, multiplier=2.0, deadline=60.0 #
    )
    def __call__(self, input_texts: Documents) -> Embeddings:
        if not input_texts:
            return []
        if not isinstance(input_texts, list) or not all(isinstance(doc, str) for doc in input_texts):
            raise TypeError("Input must be a list of document strings.")

        try:
            # Ensure task_type is set for the current operation.
            # The task_type might be changed externally before calling this for queries.
            response = genai.embed_content(
                model=self.model_name,
                content=input_texts,
                task_type=self.task_type # Use the instance's current task_type
            )
            return response['embedding'] if isinstance(response['embedding'], list) and isinstance(response['embedding'][0], list) else [response['embedding']] # Ensure list of lists for multiple inputs
        except Exception as e:
            logger.error(f"Error generating embeddings with Gemini: {e} for task_type: {self.task_type}")
            # Fallback to returning zero vectors of expected dimension if model is known, else raise
            # For text-embedding-004, dimension is 768
            if "text-embedding-004" in self.model_name:
                 return [[0.0] * 768 for _ in input_texts]
            raise  # Re-raise if it's not a known embedding error or fallback isn't appropriate

# Global ChromaDB client and collection for DE KB
db_de_kb_instance: Optional[ChromaCollectionType] = None # Use the aliased type

# Corrected function signature
def initialize_de_knowledge_base(kb_data: List[Dict[str, str]] = CAPSTONE_KB_DATA) -> Optional[ChromaCollectionType]:
    global db_de_kb_instance
    if not AGENT_CONFIG["GOOGLE_API_KEY"]:
        logger.error("Google API Key not available. Cannot initialize Gemini Embedding Function for RAG.")
        return None

    try:
        embed_fn = GeminiEmbeddingFunctionForChroma(api_key=AGENT_CONFIG["GOOGLE_API_KEY"], task_type="retrieval_document")
        chroma_client = chromadb.PersistentClient(path=AGENT_CONFIG["DB_PATH_DE_KB"])

        db_de_kb_instance = chroma_client.get_or_create_collection(
            name=AGENT_CONFIG["COLLECTION_NAME_DE_KB"],
            embedding_function=embed_fn,
            metadata={"hnsw:space": "cosine"} # Specify cosine distance
        )

        if db_de_kb_instance.count() == 0 and kb_data:
            logger.info(f"Populating Data Engineering KB in ChromaDB at {AGENT_CONFIG['DB_PATH_DE_KB']}...")
            documents_to_add = []
            ids_to_add = []
            metadatas_to_add = []
            for i, entry in enumerate(kb_data):
                doc_str = f"ID: {entry.get('id', f'doc_{i}')}\nType: {entry.get('type', 'N/A')}\nTechnology: {entry.get('technology', 'N/A')}\nDescription: {entry.get('description', '')}\nSuggestion: {entry.get('suggestion', '')}"
                documents_to_add.append(doc_str)
                ids_to_add.append(entry.get('id', f'doc_{i}')) # Must be unique
                # Add other fields as metadata for potential filtering
                meta = {k: v for k, v in entry.items() if k != 'id'}
                metadatas_to_add.append(meta)

            # Batch add
            batch_size = 100
            for i in range(0, len(documents_to_add), batch_size):
                db_de_kb_instance.add(
                    documents=documents_to_add[i:i+batch_size],
                    ids=ids_to_add[i:i+batch_size],
                    metadatas=metadatas_to_add[i:i+batch_size]
                )
                logger.info(f"Added batch of {len(documents_to_add[i:i+batch_size])} documents to KB.")
            logger.info(f"Data Engineering KB populated with {db_de_kb_instance.count()} documents.")
        else:
            logger.info(f"Data Engineering KB already exists with {db_de_kb_instance.count()} documents.")

        return db_de_kb_instance

    except Exception as e:
        logger.error(f"Failed to initialize Data Engineering Knowledge Base: {e}", exc_info=True)
        db_de_kb_instance = None
        return None

async def search_data_engineering_kb_rag(query: str, n_results: int = 3, technology_filter: Optional[str] = None) -> Optional[List[Dict[str, Any]]]:
    if not db_de_kb_instance:
        logger.error("Data Engineering KB (db_de_kb_instance) is not initialized. RAG search unavailable.")
        return None
    if not AGENT_CONFIG["GOOGLE_API_KEY"]:
        logger.error("Google API Key not available for RAG search.")
        return None

    try:
        # IMPORTANT: Set task_type to "retrieval_query" for embedding the query
        # Type casting to access _embedding_function which might be protected/private by convention
        current_embedding_function = db_de_kb_instance._embedding_function # type: ignore
        if isinstance(current_embedding_function, GeminiEmbeddingFunctionForChroma):
            current_embedding_function.task_type = "retrieval_query"
        else:
            logger.warning("Embedding function is not of type GeminiEmbeddingFunctionForChroma. Cannot dynamically set task_type for query.")


        where_filter = None
        if technology_filter:
            where_filter = {"technology": technology_filter}
            logger.info(f"RAG searching DE KB with query: '{query[:50]}...' and filter: {where_filter}")
        else:
            logger.info(f"RAG searching DE KB with query: '{query[:50]}...'")

        # ChromaDB's query is synchronous, run in a thread for async compatibility
        results = await asyncio.to_thread(
            db_de_kb_instance.query,
            query_texts=[query],
            n_results=n_results,
            include=['documents', 'metadatas', 'distances'],
            where=where_filter
        )

        # Reset task_type to default after query if needed
        if isinstance(current_embedding_function, GeminiEmbeddingFunctionForChroma):
            current_embedding_function.task_type = "retrieval_document"


        if results and results.get("documents") and results["documents"][0]:
            parsed_results = []
            for i, doc_str in enumerate(results["documents"][0]): # type: ignore
                entry = {
                    "id": results["ids"][0][i] if results.get("ids") and results["ids"][0] else f"retrieved_doc_{i}", # type: ignore
                    "document_content": doc_str,
                    "metadata": results["metadatas"][0][i] if results.get("metadatas") and results["metadatas"][0] else {}, # type: ignore
                    "distance": results["distances"][0][i] if results.get("distances") and results["distances"][0] else float('inf') # type: ignore
                }
                parsed_results.append(entry)
            logger.info(f"RAG search found {len(parsed_results)} results.")
            return parsed_results
        else:
            logger.info("RAG search found no results.")
            return []

    except Exception as e:
        logger.error(f"Error during RAG search in Data Engineering KB: {e}", exc_info=True)
        # Attempt to reset task_type in case of error
        if db_de_kb_instance and hasattr(db_de_kb_instance, '_embedding_function'):
             current_embedding_function = db_de_kb_instance._embedding_function # type: ignore
             if isinstance(current_embedding_function, GeminiEmbeddingFunctionForChroma):
                current_embedding_function.task_type = "retrieval_document"
        return None

# Initialize the KB when this "module" is loaded
db_de_kb_instance = initialize_de_knowledge_base()
logger.info("Knowledge Base Management (kb_manager) defined and KB initialized if possible.")


In [8]:
### 7. Tool Definitions (Simulated tools/*.py)

from langchain_core.tools import tool

# --- Simulated Data Sources ---
RAW_CUSTOMER_DATA_DF = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Alice Smith', 'Bob Johnson', 'Carol Williams', 'David Brown', 'Eve Davis'],
    'email': ['alice@example.com', 'bob@example.com', 'carol@example.com', 'david@example.com', 'eve@example.com'],
    'join_date': pd.to_datetime(['2023-01-15', '2023-02-20', '2023-01-10', '2023-03-01', '2023-02-05']),
    'order_count': [5, '3', 7, 2.0, 4] # Mixed type for DQ demo
})

PIPELINE_METRICS_DATA_DF = pd.DataFrame({
    'run_id': ['run_001', 'run_002', 'run_003', 'run_004', 'run_005'],
    'pipeline_name': ['customer_ingest', 'product_update', 'customer_ingest', 'sales_report', 'customer_ingest'],
    'start_time': pd.to_datetime(['2024-05-01 10:00:00', '2024-05-01 11:00:00', '2024-05-02 10:00:00', '2024-05-02 12:00:00', '2024-05-03 10:00:00']),
    'duration_seconds': [120, 300, 150, 600, 100],
    'status': ['SUCCESS', 'SUCCESS', 'FAILURE', 'SUCCESS', 'SUCCESS'],
    'records_processed': [1000, 500, 0, 50000, 1200]
})

SIMULATED_DATA_ENG_SOURCES = {
    "raw_customer_data": RAW_CUSTOMER_DATA_DF,
    "pipeline_metrics": PIPELINE_METRICS_DATA_DF
}
# --- End Simulated Data Sources ---

@tool
def query_simulated_data_source(data_source_name: Literal["raw_customer_data", "pipeline_metrics"], query_filter: Optional[str] = None) -> Dict[str, Any]:
    """
    Queries a simulated data source (Pandas DataFrame).
    Args:
        data_source_name: The name of the data source to query.
        query_filter: An optional Pandas query string (e.g., 'status == "FAILURE"').
                      If None or empty, returns a sample of the whole source.
    """
    logger.info(f"Tool 'query_simulated_data_source' called for source: '{data_source_name}' with filter: '{query_filter}'")
    if data_source_name not in SIMULATED_DATA_ENG_SOURCES:
        return {"error": f"Data source '{data_source_name}' not found.", "results_sample": []}

    df = SIMULATED_DATA_ENG_SOURCES[data_source_name].copy()
    result_df = df
    query_applied = False

    if query_filter and query_filter.strip():
        # Basic safety check (very rudimentary, for demo purposes)
        unsafe_patterns = [';', '__', 'eval(', 'exec(', 'import ']
        if any(pattern in query_filter for pattern in unsafe_patterns):
            return {"error": "Query filter contains potentially unsafe patterns.", "results_sample": []}
        try:
            result_df = df.query(query_filter, engine='python') # 'python' engine is more flexible but can be slower
            query_applied = True
        except Exception as e:
            return {"error": f"Error executing query: {str(e)}", "results_sample": []}

    sample_df = result_df.head(10) # Return a sample
    # Convert Timestamp to string for JSON serialization
    for col in sample_df.select_dtypes(include=['datetime64[ns]']).columns:
        sample_df[col] = sample_df[col].dt.strftime('%Y-%m-%d %H:%M:%S')

    return {
        "data_source_name": data_source_name,
        "query_filter_applied": query_filter if query_applied else "N/A (or entire dataset sampled)",
        "total_matching_entries": len(result_df),
        "results_sample": sample_df.to_dict(orient='records')
    }

@tool
async def analyze_data_problem_with_de_kb_rag(problem_description: str, specific_query: Optional[str] = None, technology_context: Optional[str] = None) -> Dict[str, Any]:
    """
    Analyzes a data engineering problem or context using the Data Engineering Knowledge Base (RAG).
    Args:
        problem_description: A summary of the data problem, context, or question.
        specific_query: An optional more specific question to pose to the KB.
        technology_context: Optional specific technology (e.g., MySQL, Kafka, Spark) to filter KB results.
    """
    logger.info(f"Tool 'analyze_data_problem_with_de_kb_rag' called. Problem: '{problem_description[:50]}...', Query: '{specific_query[:50]}...', Tech: {technology_context}")
    if not db_de_kb_instance: # Check if KB was initialized
         return {"status": "error", "error_message": "Knowledge Base not initialized.", "relevant_kb_entries": []}

    rag_query = f"{problem_description}\n{specific_query}" if specific_query else problem_description
    retrieved_docs = await search_data_engineering_kb_rag(query=rag_query, n_results=3, technology_filter=technology_context)

    if retrieved_docs is None: # Error in search
        return {"status": "error", "error_message": "Error during RAG search.", "relevant_kb_entries": []}
    if not retrieved_docs:
        return {"status": "success", "message": "No highly relevant entries found in KB for the query.", "relevant_kb_entries": []}

    return {"status": "success", "relevant_kb_entries": retrieved_docs}


@tool
async def generate_etl_script_tool(source_description: str, target_description: str, transformation_logic: str, script_type: Literal["python_pandas", "pyspark", "sql"] = "python_pandas") -> Dict[str, str]:
    """
    (MOCK) Generates an ETL script snippet based on descriptions.
    In a real scenario, this would call an LLM to generate the script.
    Args:
        source_description: Description of the source data/system.
        target_description: Description of the target data/system.
        transformation_logic: High-level logic for transforming source to target.
        script_type: The desired type of script (e.g., python_pandas, pyspark, sql).
    """
    logger.info(f"Tool 'generate_etl_script_tool' (MOCK) called for {script_type}.")
    # This is a MOCKED response. A real implementation would use an LLM.
    mock_script = f"# ETL Script ({script_type})\n"
    mock_script += f"# Source: {source_description}\n"
    mock_script += f"# Target: {target_description}\n"
    mock_script += f"# Transformation Logic: {transformation_logic}\n\n"

    if script_type == "python_pandas":
        mock_script += "import pandas as pd\n\n"
        mock_script += "def etl_process(source_df: pd.DataFrame) -> pd.DataFrame:\n"
        mock_script += "    # Example: Make a copy\n"
        mock_script += "    transformed_df = source_df.copy()\n"
        mock_script += "    # Apply transformations based on: {transformation_logic}\n"
        mock_script += "    # e.g., transformed_df['new_column'] = transformed_df['existing_column'] * 2\n"
        mock_script += "    print(f'Mock Pandas ETL: Processed {len(transformed_df)} rows.')\n"
        mock_script += "    return transformed_df\n"
    elif script_type == "pyspark":
        mock_script += "from pyspark.sql import SparkSession\n"
        mock_script += "from pyspark.sql.functions import col\n\n"
        mock_script += "def etl_process_spark(source_df):\n"
        mock_script += "    # Example: Select columns\n"
        mock_script += "    # transformed_df = source_df.select(col('id'), col('value'))\n"
        mock_script += "    transformed_df = source_df # Placeholder\n"
        mock_script += "    print(f'Mock PySpark ETL: Processed approx {transformed_df.count()} rows.')\n"
        mock_script += "    return transformed_df\n"
    elif script_type == "sql":
        mock_script += "/* SQL Transformation Snippet */\n"
        mock_script += "-- Assuming source_table and target_table exist\n"
        mock_script += "-- INSERT INTO target_table (col1, col2)\n"
        mock_script += "-- SELECT source_col_A, source_col_B FROM source_table WHERE ... ;\n"
        mock_script += "SELECT 'Mock SQL ETL script' AS status;\n"

    return {"status": "success_mocked", "script_type": script_type, "script": mock_script}

@tool
async def generate_pipeline_run_summary(overall_status: Literal["SUCCESS", "FAILURE", "PARTIAL_SUCCESS"],
                                     task_description: str,
                                     key_metrics: Optional[List[Dict[str, Any]]] = None,
                                     issues_found: Optional[List[str]] = None,
                                     generated_artifacts_summary: Optional[List[str]] = None,
                                     recommendations: Optional[List[str]] = None) -> Dict[str, str]:
    """Generates a final summary report for a data engineering task or pipeline run."""
    logger.info(f"Tool 'generate_pipeline_run_summary' called. Status: {overall_status}")
    report = f"# Data Pipeline Run Summary\n\n"
    report += f"**Task Description:** {task_description}\n"
    report += f"**Overall Status:** {overall_status}\n"
    report += f"**Generated At:** {datetime.datetime.now().isoformat()}\n\n"

    if key_metrics:
        report += "## Key Metrics:\n"
        for metric in key_metrics:
            report += f"- {metric.get('name', 'N/A')}: {metric.get('value', 'N/A')} {metric.get('unit', '')}\n"
        report += "\n"
    if issues_found:
        report += "## Issues Found:\n"
        for issue in issues_found:
            report += f"- {issue}\n"
        report += "\n"
    if generated_artifacts_summary:
        report += "## Generated Artifacts Summary:\n"
        for artifact in generated_artifacts_summary:
            report += f"- {artifact}\n"
        report += "\n"
    if recommendations:
        report += "## Recommendations / Next Steps:\n"
        for rec in recommendations:
            report += f"- {rec}\n"
        report += "\n"

    return {
        "status": "success",
        "report_preview": report[:500] + "...", # Truncated for brevity in state
        "full_report_text": report
    }

ALL_DE_TOOLS = [
    query_simulated_data_source,
    analyze_data_problem_with_de_kb_rag,
    generate_etl_script_tool,
    generate_pipeline_run_summary
    # Add new tools here as they are defined for the Capstone
]
logger.info(f"All Data Engineering Tools defined. Count: {len(ALL_DE_TOOLS)}")

# Simple test for a tool (optional)
# async def test_rag_tool():
#     result = await analyze_data_problem_with_de_kb_rag(problem_description="How to design MySQL schema?", technology_context="MySQL")
#     print(json.dumps(result, indent=2))
# asyncio.run(test_rag_tool())


In [9]:
### 9.1 Core Agent Nodes (core/nodes.py - Simulated) 

from langchain_core.messages import AIMessage, ToolMessage, SystemMessage, HumanMessage

# Note: llm_de_with_tools and DATA_PIPELINE_AGENT_SYSTEM_PROMPT are accessed from the llm_setup "module" (global scope)

async def data_engineering_agent_node_logic(state: DataPipelineAgentState, current_llm_with_tools, system_prompt_str) -> Dict[str, Any]:
    """Core logic for the agent node, testable and separated."""
    if current_llm_with_tools is None:
        logger.error("LLM with tools is not initialized for agent_node_logic.")
        return {"messages": [AIMessage(content='{"thoughts": "Error: LLM not initialized for this node."}')]}

    messages_for_llm = [SystemMessage(content=system_prompt_str)] + state["messages"]

    try:
        response_from_llm_obj = await current_llm_with_tools.ainvoke(messages_for_llm)
        raw_content_str = response_from_llm_obj.content
        # logger.debug(f"Raw LLM response content: {raw_content_str}")

        temp_cleaned_str = raw_content_str.strip()

        # Robustly remove common markdown fences
        if temp_cleaned_str.startswith("``````json``````"):
            temp_cleaned_str = temp_cleaned_str[len("```")]
        elif temp_cleaned_str.startswith("```"): # Handles generic ```
            temp_cleaned_str = temp_cleaned_str[len("```"):]

        if temp_cleaned_str.endswith("```"):
            temp_cleaned_str = temp_cleaned_str[:-len("```")]

        temp_cleaned_str = temp_cleaned_str.strip() # Strip again after potential fence removal

        # Specifically check for and remove a leading "json\n" or "json "
        # This handles cases where response_mime_type="application/json" adds such a prefix.
        if temp_cleaned_str.lower().startswith("json\n"):
            temp_cleaned_str = temp_cleaned_str[len("json\n"):]
        elif temp_cleaned_str.lower().startswith("json "): # Check for "json " (with a space)
            temp_cleaned_str = temp_cleaned_str[len("json "):]

        cleaned_content_str = temp_cleaned_str.strip() # Final strip
        # logger.debug(f"Cleaned LLM response content for JSON parsing: >>>{cleaned_content_str}<<<")


        parsed_output = safe_json_loads(cleaned_content_str, logger_instance=logger)

        if parsed_output and isinstance(parsed_output, dict):
            thoughts = parsed_output.get("thoughts", "")
            tool_call_data = parsed_output.get("tool_call")

            if tool_call_data and isinstance(tool_call_data, dict) and "name" in tool_call_data and "args" in tool_call_data:
                langchain_tool_call = {
                    "name": tool_call_data["name"],
                    "args": tool_call_data["args"],
                    "id": f"tool_call_{int(time.time() * 1000)}_{tool_call_data['name']}" # More unique ID
                }
                ai_message_with_tool = AIMessage(content=thoughts, tool_calls=[langchain_tool_call])
                # logger.info(f"Agent node: LLM decided to call tool: {tool_call_data['name']}")
                return {"messages": [ai_message_with_tool]}
            else:
                # logger.info(f"Agent node: LLM provided thoughts: {thoughts[:100]}...")
                return {"messages": [AIMessage(content=json.dumps({"thoughts": thoughts}))]}
        else:
            if hasattr(response_from_llm_obj, "tool_calls") and response_from_llm_obj.tool_calls:
                # logger.info("Agent node: LLM used LangChain's direct tool_calls attribute.")
                return {"messages": [response_from_llm_obj]}
            else:
                logger.warning(f"Agent node: LLM response was not the expected JSON structure and no direct tool_calls. Cleaned string for parsing was: >>>{cleaned_content_str[:200]}<<<")
                return {"messages": [AIMessage(content=json.dumps({"thoughts": f"Received non-standard response. Could not parse as JSON. Content started with: {cleaned_content_str[:100]}"}))]}

    except Exception as e:
        logger.error(f"Error in data_engineering_agent_node_logic: {e}", exc_info=True)
        return {"messages": [AIMessage(content=json.dumps({"thoughts": f"Critical Error in agent node: {str(e)}"}))]}

async def data_engineering_agent_node(state: DataPipelineAgentState) -> Dict[str, Any]:
    """Wrapper for the agent node logic to be used in the graph."""
    return await data_engineering_agent_node_logic(state, llm_de_with_tools, DATA_PIPELINE_AGENT_SYSTEM_PROMPT)


async def update_de_state_node(state: DataPipelineAgentState) -> Dict[str, Any]:
    """Processes tool outputs and updates the agent's state."""
    updated_state_payload = {
        # Ensure we're working with copies to avoid modifying state in place if not intended before return
        "discovered_insights": list(state.get("discovered_insights", [])),
        "generated_artifacts": list(state.get("generated_artifacts", [])),
        "executed_steps": list(state.get("executed_steps", []))
    }
    # Check if 'finished' exists, if not, don't add it unless explicitly set
    if 'finished' in state:
        updated_state_payload['finished'] = state['finished']


    last_message = state["messages"][-1] if state["messages"] else None

    if not last_message or not isinstance(last_message, ToolMessage):
        logger.debug("Update_state_node: Last message not a ToolMessage or no messages. No state update from tool.")
        return {}

    tool_output_str = last_message.content
    tool_name = getattr(last_message, 'name', 'unknown_tool')
    tool_call_id = getattr(last_message, 'tool_call_id', None)

    logger.info(f"Update_state_node: Processing output from tool '{tool_name}'.")

    tool_output = safe_json_loads(tool_output_str, logger_instance=logger)
    if tool_output is None:
        tool_output = {"error": "Failed to parse tool output as JSON", "raw_content": tool_output_str}

    insight_or_artifact = {"tool_name": tool_name, "output": tool_output, "timestamp": datetime.datetime.now().isoformat()}

    if "error" in tool_output or tool_output.get("status") == "error": # Check for status key from tools
        updated_state_payload["discovered_insights"].append(insight_or_artifact)
    elif tool_name == "generate_pipeline_run_summary":
        updated_state_payload["finished"] = True # Explicitly set finished to True
        updated_state_payload["pipeline_plan"] = []
        updated_state_payload["generated_artifacts"].append(insight_or_artifact)
    elif tool_name in ["generate_etl_script_tool", "detect_data_anomalies_tool", "discover_and_infer_metadata_tool"]:
        updated_state_payload["generated_artifacts"].append(insight_or_artifact)
    else:
        updated_state_payload["discovered_insights"].append(insight_or_artifact)

    updated_state_payload["executed_steps"].append({
        "tool_name": tool_name,
        "tool_call_id": tool_call_id,
        "status": "processed_output" if "error" not in tool_output and tool_output.get("status") != "error" else "processed_error_output",
        "summary": str(tool_output)[:200] + "..."
    })
    return updated_state_payload


def decide_after_de_agent(state: DataPipelineAgentState) -> Literal["tools", "__end__"]:
    """Conditional routing: after agent node, decide if to call tools or end."""
    last_message = state["messages"][-1] if state["messages"] else None
    if last_message and isinstance(last_message, AIMessage):
        if hasattr(last_message, "tool_calls") and last_message.tool_calls and len(last_message.tool_calls) > 0:
            # Ensure tool_calls[0] is a dict with 'name' and 'args' (LangChain ToolCall type)
            # or our custom dict structure that was transformed.
            first_tool_call = last_message.tool_calls[0]
            if isinstance(first_tool_call, dict) and 'name' in first_tool_call and 'args' in first_tool_call:
                 if state.get("iterations", 0) < AGENT_CONFIG["MAX_ITERATIONS_DE"]:
                    # logger.info("Routing: Agent -> Tools")
                    return "tools"
                 else:
                    logger.warning("Routing: Agent -> End (Max Iterations Reached before tool call)")
                    return "__end__"
            else: # Malformed tool_call
                logger.warning(f"Routing: Agent -> End (Malformed tool_call in AIMessage: {first_tool_call})")
                return "__end__"
        else:
            logger.info("Routing: Agent -> End (No valid tool call from LLM)")
            return "__end__"
    logger.warning("Routing: Agent -> End (Last message not AIMessage or unexpected state)")
    return "__end__"


def decide_after_de_update(state: DataPipelineAgentState) -> Literal["agent", "__end__"]:
    """Conditional routing: after state update from tools, decide if to loop to agent or end."""
    if state.get("finished", False): # Check if 'finished' key exists and is True
        logger.info("Routing: Update -> End (Finished flag is True)")
        return "__end__"
    if state.get("iterations", 0) >= AGENT_CONFIG["MAX_ITERATIONS_DE"]: # Iterations check
        logger.warning("Routing: Update -> End (Max Iterations Reached after update)")
        return "__end__"
    # logger.info("Routing: Update -> Agent (Continuing workflow)")
    return "agent"

logger.info("Core Agent Nodes (logic, update, routing) defined.")


In [10]:
### 9.2 Core LLM Setup (core/llm_setup.py - Simulated)

from langchain_google_genai import ChatGoogleGenerativeAI # [1, 2, 3, 4]
import datetime # For including current date in the prompt

# This will be populated by initialize_llm_with_tools
llm_de_with_tools = None # Global variable to hold the LLM instance with tools bound

# System prompt for the Data Engineering Agent
# AGENT_CONFIG should be defined in a previous cell (Cell 5 in the full refactored notebook)
# Example: AGENT_CONFIG = {"LLM_MODEL_NAME": "gemini-2.0-flash", "LLM_TEMPERATURE": 0.2}

DATA_PIPELINE_AGENT_SYSTEM_PROMPT = f"""You are an autonomous Data Engineering AI Agent (v2.0 - Capstone Edition). Your primary goal is to assist with Data Engineering tasks by understanding user goals, planning steps, and utilizing available tools effectively.

You operate in a cycle:
1.  **Think**: Based on the current goal, conversation history, and available tools, decide the best next step. Your thoughts should be clear and justify your chosen action or tool call.
2.  **Act**: If you decide to use a tool, you MUST format your response as a JSON object containing a `tool_call` key. The `tool_call` object must have a `name` (the tool name) and `args` (a dictionary of arguments for the tool).
    Example for tool call:
    ```
    {{
      "thoughts": "I need to query the customer data to understand its structure before generating an ETL script. I will use the query_simulated_data_source tool.",
      "tool_call": {{
        "name": "query_simulated_data_source",
        "args": {{
          "data_source_name": "raw_customer_data",
          "query_filter": "order_count > 5"
        }}
      }}
    }}
    ```
3.  **Observe**: After a tool is executed, you will receive its output. Analyze this output to inform your next thought process.

**Available Tools Overview:**
(The system will bind the actual tools, but be aware of their general capabilities from their names and descriptions if provided by the user or in previous turns.)

**Workflow and State Management:**
- You are part of a stateful graph. Your responses update the 'messages' in the state.
- Refer to 'initial_goal', 'current_task_description', 'discovered_insights', 'generated_artifacts', 'pipeline_plan', and 'executed_steps' from the state if they provide useful context (though you don't directly receive the full state object, your reasoning should be consistent with a persistent state).
- If you believe the goal is complete or cannot proceed further, use the `generate_pipeline_run_summary` tool to provide a final report. This tool call will typically end the current workflow.
- If you need more information from the user, ask clear questions in your 'thoughts' and do not call a tool. (For this agent, we are focusing on tool use primarily).
- Be methodical. Break down complex goals into smaller, manageable steps.
- If a tool fails or returns an error, analyze the error and decide if you can retry with different parameters, use a different tool, or conclude that the task cannot be completed.

**Output Format (CRITICAL):**
- Your primary output MUST be a JSON object.
- When calling a tool, the JSON must include "thoughts" and "tool_call" keys as shown above.
- If you are not calling a tool (e.g., providing a final thought before summary, or if you were designed to ask questions), your JSON output should contain only the "thoughts" key:
{{
  "thoughts": "I have analyzed the results and now I understand X. My next step should be Y."
}}
- Ensure your JSON is always valid.

**Data Engineering Capstone Context:**
This agent is intended to assist with a Data Engineering Capstone project involving MySQL (OLTP), MongoDB (Catalog), a Data Warehouse (PostgreSQL/DB2), Airflow, Kafka, and SparkML. Tailor your assistance and tool usage (especially RAG queries) to this context when appropriate.
The Knowledge Base (RAG tool) contains specific information related to these technologies and DE best practices. Use it to inform your decisions and provide more accurate assistance. For example, if asked about schema design, query the RAG with technology context.
Today's date is: {datetime.date.today().isoformat()}
"""

def initialize_llm_with_capstone_tools(tools_list: List) -> Optional[ChatGoogleGenerativeAI]:
  global llm_de_with_tools # Allow modification of the global variable

  if not AGENT_CONFIG["GOOGLE_API_KEY"]:
      logger.error("Google API Key not available. Cannot initialize LLM.")
      llm_de_with_tools = None
      return None
  try:
      llm_instance = ChatGoogleGenerativeAI(
          model=AGENT_CONFIG["LLM_MODEL_NAME"],
          temperature=AGENT_CONFIG["LLM_TEMPERATURE"],
          convert_system_message_to_human=False, # Gemini models generally handle system messages well
          generation_config={"response_mime_type": "application/json"} # CRITICAL for structured output
      )
      llm_de_with_tools = llm_instance.bind_tools(tools_list)
      logger.info(f"LLM '{AGENT_CONFIG['LLM_MODEL_NAME']}' initialized and bound with {len(tools_list)} tools.")
      return llm_de_with_tools
  except Exception as e:
      logger.error(f"Error initializing LLM: {e}", exc_info=True)
      llm_de_with_tools = None
      return None

logger.info("Core LLM Setup defined.")
# LLM will be initialized later, once tools are fully collected.


In [11]:
### 10. Core Graph Compilation (core/graph_compiler.py - Simulated)

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# Node functions are imported from the "core_nodes_module" (defined in previous cell)
# State definition is from "core_state_module" (defined in earlier cell)

def compile_data_engineering_agent_graph(tools_list: List) -> StateGraph:
    """Compiles and returns the LangGraph StateGraph for the DE agent."""
    if not tools_list:
        logger.warning("Compiling agent graph with an empty tools list. Agent will not be able to use tools.")

    de_graph_builder = StateGraph(DataPipelineAgentState)

    # Add nodes
    de_graph_builder.add_node("agent", data_engineering_agent_node) # Defined in core_nodes
    tools_executor_node = ToolNode(tools_list) # LangGraph's prebuilt tool executor
    de_graph_builder.add_node("tools", tools_executor_node)
    de_graph_builder.add_node("update_state", update_de_state_node) # Defined in core_nodes

    # Define edges
    de_graph_builder.add_edge(START, "agent")
    de_graph_builder.add_conditional_edges(
        "agent",
        decide_after_de_agent, # Defined in core_nodes
    )
    de_graph_builder.add_edge("tools", "update_state")
    de_graph_builder.add_conditional_edges(
        "update_state",
        decide_after_de_update, # Defined in core_nodes
    )

    logger.info("Data Engineering Agent graph definition complete.")
    return de_graph_builder.compile()

logger.info("Core Graph Compilation logic defined.")


In [12]:
### 11. Main Agent Orchestration & Execution

# This cell acts like the main `run_agent.py` or orchestrator script.

data_pipeline_agent_app = None # Will hold the compiled graph

def initialize_full_agent_system():
    global data_pipeline_agent_app
    global llm_de_with_tools # from llm_setup

    logger.info("Initializing full Data Engineering Agent system...")

    # 1. Load configurations (already done into AGENT_CONFIG)
    logger.info(f"Agent Config: {AGENT_CONFIG}")

    # 2. Initialize KB (already done on kb_manager "module" load if successful)
    if db_de_kb_instance:
        logger.info("Knowledge Base appears to be initialized.")
    else:
        logger.warning("Knowledge Base initialization failed or skipped. RAG tool may not work.")

    # 3. Collect tools (ALL_DE_TOOLS is a global list from the tools "module")
    if not ALL_DE_TOOLS:
        logger.error("No tools found in ALL_DE_TOOLS. Agent will be severely limited.")
        return False
    logger.info(f"Collected {len(ALL_DE_TOOLS)} tools.")

    # 4. Initialize LLM with these tools
    # The initialize_llm_with_capstone_tools function is from llm_setup "module"
    # It populates the global llm_de_with_tools
    initialized_llm = initialize_llm_with_capstone_tools(ALL_DE_TOOLS)
    if not initialized_llm:
        logger.error("Failed to initialize LLM with tools. Agent cannot run.")
        return False

    # 5. Compile the agent graph
    # The compile_data_engineering_agent_graph function is from graph_compiler "module"
    try:
        data_pipeline_agent_app = compile_data_engineering_agent_graph(ALL_DE_TOOLS)
        logger.info("Data Engineering Agent graph compiled successfully.")
    except Exception as e:
        logger.error(f"Failed to compile agent graph: {e}", exc_info=True)
        return False

    logger.info("Full Data Engineering Agent system initialization complete.")
    return True


async def execute_data_pipeline_workflow(initial_goal: str, recursion_limit: int = 25):
    """
    Executes the data engineering workflow for a given goal.
    """
    if not data_pipeline_agent_app:
        logger.error("Agent graph (data_pipeline_agent_app) is not compiled. Cannot execute workflow.")
        print("Error: Agent not initialized. Please run `initialize_full_agent_system()` first.")
        return None
    if not llm_de_with_tools: # from llm_setup
        logger.error("LLM (llm_de_with_tools) is not initialized. Cannot execute workflow.")
        print("Error: LLM not initialized.")
        return None


    logger.info(f"\n--- Starting New Workflow --- \nGoal: {initial_goal}\n---------------------------")

    # Initial state for the graph
    current_run_state = DataPipelineAgentState(
        messages=[HumanMessage(content=initial_goal)], # Start with the human goal
        initial_goal=initial_goal,
        current_task_description=initial_goal, # Initial task is the goal itself
        discovered_insights=[],
        generated_artifacts=[],
        pipeline_plan=[], # Agent might populate this
        executed_steps=[],
        finished=False,
        iterations=0
    )

    final_state = None
    graph_config = {"recursion_limit": recursion_limit}

    # Main execution loop (LangGraph handles internal looping based on conditional edges)
    # We're essentially stepping through the graph using ainvoke until it reaches an END state or recursion limit
    try:
        async for event_output in data_pipeline_agent_app.astream(current_run_state, config=graph_config):
            # event_output is a dictionary where keys are node names and values are the output of that node (state update)
            # We can inspect these events if needed for detailed logging or updates
            # For simplicity here, we'll just get the final state after the stream ends.
            # The last event for a given node contains its final output for that invocation.
            # The stream will end when an "__end__" node is reached or recursion limit.

            # The `astream` method yields the state *after* each node has executed.
            # The final `event_output` from the stream will contain the state from the last executed node before hitting END.
            # LangGraph's `astream` yields the full state after each node, so the "final_state" will be the one
            # just before termination, or the state when END is hit.
            # The key is that the state IS the message stream.

            # Let's log the output of each node
            for node_name, node_state_update in event_output.items():
                logger.info(f"Output from node '{node_name}':")
                # The node_state_update is the new state *after* this node ran.
                # We are interested in the latest message added.
                if node_state_update and "messages" in node_state_update and node_state_update["messages"]:
                    last_msg_in_update = node_state_update["messages"][-1]
                    if isinstance(last_msg_in_update, AIMessage):
                        display(Markdown(f"**🤖 Agent Thought/Action ({node_name}):**"))
                        if last_msg_in_update.tool_calls:
                             display(Markdown(f"> _{last_msg_in_update.content}_ \n> Tool Call: `{last_msg_in_update.tool_calls[0]['name']}` with args `{last_msg_in_update.tool_calls[0]['args']}`"))
                        else:
                             display(Markdown(f"> _{last_msg_in_update.content}_")) # Should be JSON thoughts
                    elif isinstance(last_msg_in_update, ToolMessage):
                        display(Markdown(f"**🛠️ Tool Result ({node_name} for tool '{last_msg_in_update.name}'):**"))
                        try:
                            tool_output_json = json.loads(last_msg_in_update.content)
                            display(Markdown(f"``````"))
                        except json.JSONDecodeError:
                             display(Markdown(f"> {last_msg_in_update.content}"))
                # Update the overall current_run_state with the latest full state from the event
                current_run_state = node_state_update # The event_output[node_name] is the full state after that node

        final_state = current_run_state # The state after the stream has completed

    except Exception as e:
        logger.error(f"Error during agent workflow execution: {e}", exc_info=True)
        print(f"Workflow execution failed: {e}")
        final_state = current_run_state # Capture state at point of failure
        final_state["messages"].append(AIMessage(content=json.dumps({"thoughts": f"Workflow ended due to error: {e}"})))


    logger.info("--- Workflow Ended ---")
    if final_state:
        display(Markdown("## Final Workflow State:"))
        display(Markdown(f"**Initial Goal:** {final_state.get('initial_goal')}"))
        display(Markdown(f"**Finished:** {final_state.get('finished')}"))
        display(Markdown(f"**Iterations to reach END (LangGraph internal):** (Graph manages iterations to END, `final_state['iterations']` is our manual counter if we were looping `ainvoke`)")) # The 'iterations' in state is for our own loop if we had one. LangGraph's stream ends when it hits END.

        display(Markdown("### Discovered Insights:"))
        if final_state.get("discovered_insights"):
            for insight in final_state["discovered_insights"]:
                display(Markdown(f"- Tool: `{insight.get('tool_name', 'N/A')}` Output: \n  ``````"))
        else:
            display(Markdown("_No insights discovered._"))

        display(Markdown("### Generated Artifacts:"))
        if final_state.get("generated_artifacts"):
            for artifact_info in final_state["generated_artifacts"]: # artifact_info structure {tool_name, output}
                display(Markdown(f"- Tool: `{artifact_info.get('tool_name')}`"))
                if artifact_info.get("tool_name") == "generate_pipeline_run_summary":
                     display(Markdown(f"  **Summary Report:**\n``````"))
                elif artifact_info.get("tool_name") == "generate_etl_script_tool":
                     display(Markdown(f"  **Generated Script ({artifact_info['output'].get('script_type')}):**\n``````"))
                else:
                     display(Markdown(f"  Output: \n  ``````"))
        else:
            display(Markdown("_No artifacts generated._"))

        # Display conversation history
        display(Markdown("### Conversation History:"))
        for msg in final_state.get("messages", []):
            if isinstance(msg, HumanMessage):
                display(Markdown(f"**👤 User:** {msg.content}"))
            elif isinstance(msg, AIMessage):
                if msg.tool_calls:
                    display(Markdown(f"**🤖 Agent:** _{msg.content}_ \n CALLING TOOL: `{msg.tool_calls[0]['name']}` with args `{msg.tool_calls[0]['args']}`"))
                else: # Should be thoughts
                    try:
                        thought_json = json.loads(msg.content)
                        display(Markdown(f"**🤖 Agent:** _{thought_json.get('thoughts', msg.content)}_"))
                    except:
                        display(Markdown(f"**🤖 Agent:** _{msg.content}_")) # Fallback
            elif isinstance(msg, ToolMessage):
                display(Markdown(f"**🛠️ Tool ({msg.name}):** Output received (see details above or in insights/artifacts)."))
            elif isinstance(msg, SystemMessage):
                 display(Markdown(f"**⚙️ System:** _{msg.content[:100]}_..."))


    return final_state


# Initialize the system when this cell is run
if GOOGLE_API_KEY:
    initialization_success = initialize_full_agent_system()
    if initialization_success:
        logger.info("Agent system ready.")
        # Visualize graph (optional, requires graphviz)
        # try:
        #     img_bytes = data_pipeline_agent_app.get_graph().draw_mermaid_png()
        #     display(Image(img_bytes))
        # except Exception as e:
        #     logger.warning(f"Could not draw graph: {e}. Ensure graphviz is installed if you want to visualize.")
    else:
        logger.error("Agent system initialization failed. Please check logs.")
else:
    logger.error("GOOGLE_API_KEY not found. Agent cannot be initialized.")

logger.info("Main Agent Orchestration and Execution logic defined.")


/tmp/ipykernel_99/3392808605.py:36: UserWarning: WARNING! generation_config is not default parameter.
                generation_config was transferred to model_kwargs.
                Please confirm that generation_config is what you intended.
  initialized_llm = initialize_llm_with_capstone_tools(ALL_DE_TOOLS)


In [13]:
### 12. Example Capstone Workflows

async def run_capstone_scenarios():
    if not data_pipeline_agent_app or not llm_de_with_tools:
        print("Agent not initialized. Cannot run scenarios. Check previous cell output for errors.")
        logger.error("Attempted to run scenarios but agent is not ready.")
        return
        
    # Fix: Check KB status before running scenarios
    global db_de_kb_instance
    if not db_de_kb_instance:
        print("WARNING: Knowledge Base is not initialized. Attempting to initialize now.")
        db_de_kb_instance = initialize_de_knowledge_base()
        if db_de_kb_instance:
            print(f"Knowledge Base initialized successfully. Document count: {db_de_kb_instance.count()}")
        else:
            print("WARNING: Knowledge Base initialization failed. RAG functionality will be limited.")

    # --- Define Capstone Project Goals ---
    goal1_mysql_schema_help = (
        "I'm in Module 1 of my Data Engineering Capstone. "
        "I need to design a MySQL OLTP schema for e-commerce with customer and order tables. "
        "Can you use the RAG tool to find best practices for this, specifically for MySQL technology?"
    )

    goal2_etl_script_draft = (
        "For Module 3-b (DWH Implementation), I need to load data from a simulated 'raw_customer_data' source into a dimension table. "
        "The source has customer_id, name, email, join_date, order_count. "
        "The target is a customer dimension. Transformations needed: standardize email to lowercase, combine 'name' into 'full_name'. "
        "Generate a Python Pandas ETL script for this."
    )

    goal3_airflow_guidance_and_summary = (
        "I'm working on ETL pipelines with Airflow (Module 5). "
        "Use the RAG to find best practices for Airflow task idempotency. "
        "Then, provide a summary report for this task, assuming it was successful and the key finding was related to idempotency."
    )

    # --- Run Scenarios ---
    print("\n--- Running Scenario 1: MySQL Schema Help ---")
    await execute_data_pipeline_workflow(initial_goal=goal1_mysql_schema_help)

    print("\n--- Running Scenario 2: ETL Script Draft ---")
    await execute_data_pipeline_workflow(initial_goal=goal2_etl_script_draft)

    print("\n--- Running Scenario 3: Airflow Guidance & Summary ---")
    await execute_data_pipeline_workflow(initial_goal=goal3_airflow_guidance_and_summary)

# This ensures the asyncio event loop is handled correctly in Jupyter
if GOOGLE_API_KEY and data_pipeline_agent_app: # Only run if agent initialized
    import nest_asyncio
    nest_asyncio.apply() # Allow re-entrant asyncio loops in Jupyter
    
    # Fix: Force KB reinitialization before running scenarios
    db_de_kb_instance = None
    db_de_kb_instance = initialize_de_knowledge_base()
    if db_de_kb_instance:
        print(f"Knowledge Base initialized successfully. Document count: {db_de_kb_instance.count()}")
    else:
        print("WARNING: Knowledge Base initialization failed. RAG functionality will be limited.")
        
    asyncio.run(run_capstone_scenarios())
else:
    print("Skipping example workflows as agent is not initialized (check GOOGLE_API_KEY or initialization logs).")

logger.info("Example Capstone Workflows cell execution finished.")


Knowledge Base initialized successfully. Document count: 6

--- Running Scenario 1: MySQL Schema Help ---


**🛠️ Tool Result (Tool: 'analyze_data_problem_with_de_kb_rag'):**

> content='{"status": "success", "relevant_kb_entries": [{"id": "dq_rule_mysql_1", "document_content": "ID: dq_rule_mysql_1\\nType: Data Quality Rule\\nTechnology: MySQL\\nDescription: Ensure primary keys in MySQL OLTP tables are not null and unique.\\nSuggestion: Define columns as PRIMARY KEY. For existing data, check with `COUNT(DISTINCT pk_column) = COUNT(*)` and `SUM(IF(pk_column IS NULL, 1, 0)) = 0`.", "metadata": {"description": "Ensure primary keys in MySQL OLTP tables are not null and unique.", "suggestion": "Define columns as PRIMARY KEY. For existing data, check with `COUNT(DISTINCT pk_column) = COUNT(*)` and `SUM(IF(pk_column IS NULL, 1, 0)) = 0`.", "technology": "MySQL", "type": "Data Quality Rule"}, "distance": 0.4208086133003235}]}' name='analyze_data_problem_with_de_kb_rag' tool_call_id='tool_call_1747258489608_analyze_data_problem_with_de_kb_rag'

**🛠️ Tool Result (Tool: 'analyze_data_problem_with_de_kb_rag'):**

> content='{"status": "success", "relevant_kb_entries": [{"id": "dq_rule_mysql_1", "document_content": "ID: dq_rule_mysql_1\\nType: Data Quality Rule\\nTechnology: MySQL\\nDescription: Ensure primary keys in MySQL OLTP tables are not null and unique.\\nSuggestion: Define columns as PRIMARY KEY. For existing data, check with `COUNT(DISTINCT pk_column) = COUNT(*)` and `SUM(IF(pk_column IS NULL, 1, 0)) = 0`.", "metadata": {"description": "Ensure primary keys in MySQL OLTP tables are not null and unique.", "type": "Data Quality Rule", "technology": "MySQL", "suggestion": "Define columns as PRIMARY KEY. For existing data, check with `COUNT(DISTINCT pk_column) = COUNT(*)` and `SUM(IF(pk_column IS NULL, 1, 0)) = 0`."}, "distance": 0.4528235197067261}]}' name='analyze_data_problem_with_de_kb_rag' tool_call_id='tool_call_1747258491110_analyze_data_problem_with_de_kb_rag'

...

### Generated Artifacts:

_No artifacts generated._

### Full Conversation History (from final state):

**👤 User:** I'm working on ETL pipelines with Airflow (Module 5). Use the RAG to find best practices for Airflow task idempotency. Then, provide a summary report for this task, assuming it was successful and the key finding was related to idempotency.

**🤖 Agent:** _Critical Error in agent node: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 34
}
]_